# Tables 1-4: Creating tables

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# @author: Sebastian Gegenleithner, Manuel Pirker

## Imports

In [2]:
import tensorflow as tf

import os

from ForecastModel.utils.metrics import calculate_nse, calculate_kge, calculate_bias

import numpy as np
import pandas as pd

import json

## Functions

In [3]:
# helper functions to create latex tables
def load_metrics(path):
    with open(path, "r") as f:
        metrics = json.load(f)
    return metrics

def df2latex(df, file, bold_mask=None, fmt=None):
    print(file)
    if type(bold_mask) == type(None):
        bold_mask = np.zeros(df.shape)
    
    with open(file, "w") as f:
        f.write("index&"+"&".join(df.columns)+r"\\")
        f.write("\n")
        i = -1
        for n, row in df.iterrows():
            i += 1
            f.write(str(n))
            for j,num in enumerate(row.values):
                if bold_mask[i,j] == 1:
                    f.write(r"&\textbf{")
                else:
                    f.write(r"&")
                if type(fmt) == type(None):
                    if np.abs(num) >= 100:
                        f.write(f"{int(num): 6d}")
                    elif np.abs(num) >= 10:
                        f.write(f"{num: 6.1f}")
                    elif np.abs(num) >= 1:
                        f.write(f"{num: 6.2f}")
                    else:
                        f.write(f"{num: 6.3f}")
                elif type(fmt) == type([]):
                    if fmt[j] == 'd':
                       num = int(num)
                    f.write(format(num, fmt[j]))
                else:
                    if fmt == 'd':
                        num = int(num)
                    f.write(format(num, fmt))
                if bold_mask[i,j] == 1:
                    f.write(r"}")
                    
            f.write(r"\\")
            f.write("\n")

def get_bold_mask(df, fcn=np.argmax, n_multi_cols=3, offset=0):
    mask = np.zeros(df.shape)
    for n in range(n_multi_cols):
        if type(fcn) == type([]):
            idx = fcn[n](df.values[:, (offset+n)::n_multi_cols], axis=1)
        else:
            idx = fcn(df.values[:, (offset+n)::n_multi_cols], axis=1)
        for j in range(df.shape[0]):
            mask[j, idx[j]*n_multi_cols+n+offset] = 1
    return mask

# other functions
def peak_loss(y, y_hat):
    mask = y >= tf.math.reduce_max(y)*0.75
    
    squared_diff  = (y - y_hat)**2

    squared_diff = tf.where(mask, 2 * squared_diff, squared_diff)

    return tf.reduce_mean(squared_diff)

def dt(dates_string, format="%d/%m/%Y %H:%M"):
    return pd.to_datetime(dates_string, format=format)


## Create model class

In [4]:
class Model:
    def __init__(self, name, model_folder, n_trial=-1, target_name="", feat_hindcast=[], feat_forecast=[], is_external_model= False, is_final_model= False, color="r", ls="-"):
        self.name  = name
        self.color = color
        self.ls    = ls
        self.is_external_model = is_external_model
        
        if is_final_model:
            self.lg_path = model_folder
            self.hp_path = model_folder
            if is_external_model:
                is_external_model
                # do nothing
            else:
                with open(os.path.join(self.lg_path, "features.txt"), "r") as f:
                    dic = json.load(f)
                self.target_name   = dic["target_name"]
                self.feat_hindcast = dic["feat_hindcast"]
                self.feat_forecast = dic["feat_forecast"]
        else:
            self.lg_path = os.path.join(model_folder, "logs", f"trial_{n_trial:02d}")
            self.hp_path = os.path.join(model_folder,  "hp", f"trial_{n_trial:02d}")
            self.target_name   = target_name
            self.feat_hindcast = feat_hindcast
            self.feat_forecast = feat_forecast

models = {
    "arima": Model("ARIMA",
                r"rst\ARIMA",
                is_final_model = True,
                is_external_model = True,
                color = "k",
                ls = "--",
                  ),
    "lstm_pbhm": Model("HLSTM-PBHM",
                           r"rst\HLSTM-PBHM",
                           is_final_model = True,
                           color = "#a65628",
                           ls = "-",
                          ),
     "lstm": Model("HLSTM",
                   r"rst\HLSTM",
                   is_final_model = True,
                   color = "#ff7f00",
                   ls = "-",
                 ),
     }


## Init

In [5]:
# define paths
PLOT_PATH = r"plots"
DATA_PATH = r"data\Dataset.csv"
CROSS_INDICES_PATH = r"data\indices"


## Table 1: Statistics of the catchment

In [6]:
# calculate metrics of hydrologic model per fold
dfh = pd.read_csv(DATA_PATH, parse_dates=["time"])

hyd_metrics = {}
for year in range(2011, 2018):
      dfp = dfh.loc[dfh["time"].dt.year == year]
      hyd_metrics[year] = {
          "kge":  calculate_kge(dfp.qmeasval.values, dfp.qsim.values),
          "nse":  calculate_nse(dfp.qmeasval.values, dfp.qsim.values),
          "bias": calculate_bias(dfp.qmeasval.values, dfp.qsim.values),
          "q_mean": dfp.qmeasval.mean(),
          "q_std":  dfp.qmeasval.std(),
          "q_max":  dfp.qmeasval.max(),
          "q_sum":  dfp.qmeasval.sum()*0.25*60*60 / 1e6,
          "qs_mean": dfp.qsim.mean(),
          "qs_std":  dfp.qsim.std(),
          "qs_max":  dfp.qsim.max(),
          "qs_sum":  dfp.qsim.sum()*0.25*60*60 / 1e6,
          "pmx_max": dfp.pmax.max()*4,
          "pmx_sum": dfp.pmax.sum(),
          "p_max":   dfp.pmean.max()*4,
          "p_sum":   dfp.pmean.sum(),
          "t_mean": dfp.tmean.mean(),
          "t_std":  dfp.tmean.std(),
      }    

# catchment statistics
df_catchment_stats = pd.DataFrame(hyd_metrics).transpose()
df_catchment_stats = df_catchment_stats[df_catchment_stats.columns[3:]].transpose()
df_catchment_stats.columns = [str(x) for x in df_catchment_stats.columns]
df_catchment_stats

,2011,2012,2013,2014,2015,2016,2017
q_mean,0.571149,1.009310,1.208354,1.167666,0.711204,0.829312,0.567430
q_std,0.245171,0.842681,0.681078,0.669319,0.332219,0.682278,0.229892
q_max,9.610000,25.200000,15.000000,7.270000,5.850000,17.940000,9.210000
q_sum,18.011757,31.916797,38.106657,36.823518,22.428531,26.224821,17.698878
qs_mean,0.617823,1.103376,1.399735,1.349319,0.755306,0.916051,0.907808
qs_std,0.331923,0.832882,1.016826,0.719722,0.519150,0.701360,0.479473
qs_max,4.195333,8.937861,11.354018,7.681351,4.502031,6.426838,4.611206
qs_sum,19.483660,34.891398,44.142029,42.552124,23.819337,28.967737,28.315708
pmx_max,118.720001,180.520004,100.480003,84.639999,109.919998,231.880005,173.360001
pmx_sum,2159.559998,2877.670002,2777.400000,2847.860000,2198.090000,3222.800001,3076.049996


In [7]:
# write to txt file in latex format
df2latex(df_catchment_stats, os.path.join(PLOT_PATH, r"table_1_summary_data.txt"))

plots\table_1_summary_data.txt


## Table 2: Average Model Performance

In [8]:
metric_names = ["kge", "nse", "bias"]
metric_labels = ["KGE", "NSE", "PBIAS"]

xx = np.arange(1,97)
df = pd.DataFrame(columns=["name", "year"])

n_row = -5

for n, key in enumerate(models.keys()):
    metrics          = load_metrics(os.path.join(models[key].lg_path, "metrics.txt"))
    metrics_baseline = load_metrics(os.path.join(models["arima"].lg_path, "metrics.txt"))
    
    n_row += 5
    
    for j, met in enumerate(metric_names):

        metric_valid    = metrics["valid"][met]
        metric_test     = metrics["test"][met]
        metric_baseline = metrics_baseline["test"][met]
        
        for i in range(5):
            df.loc[n_row+i, ["name", "year"]] = [models[key].name, 2013+i]

            # evaluations --------------------------------------------------------------------------
            # median metrics of validation and test set
            df.loc[n_row+i, [f"{met}_valid"]]  = [np.median(metric_valid[i])]
            df.loc[n_row+i, [f"{met}_test"]]   = [np.median(metric_test[i])]
            # generalization measure
            df.loc[n_row+i, [f"{met}_general"]]= [np.median(metric_valid[i]) - np.median(metric_test[i])]
            # metric for inital and final step in forecasting window
            df.loc[n_row+i, [f"{met}_init"]]   = [metric_test[i][0]]
            df.loc[n_row+i, [f"{met}_final"]]  = [ metric_test[i][-1]]
            # difference of metric between inital and final step in forecasting window
            df.loc[n_row+i, [f"{met}_drop"]]   = [metric_test[i][0] - metric_test[i][-1]]
            # num of lead time steps required so that model outperforms the baseline (ARIMA)
            if met != "bias":
                out_perform_idxs = [n for n, (x,y) in enumerate(zip(metric_baseline[i], metric_test[i])) if y-x > 0]
                if len(out_perform_idxs) > 0:
                    df.loc[n_row+i, [f"{met}_out"]] = out_perform_idxs[0]
                    df.loc[n_row+i, [f"{met}_out_hours"]] = out_perform_idxs[0] * 0.25
                else:
                    df.loc[n_row+i, [f"{met}_out"]] = -1
                    df.loc[n_row+i, [f"{met}_out_hours"]] = -1

# summary
cols = df.columns.to_list()
df[cols[:2]+cols[4:9]+cols[12:17]+cols[20:]]

,name,year,kge_general,kge_init,kge_final,kge_drop,kge_out,nse_general,nse_init,nse_final,nse_drop,nse_out,bias_general,bias_init,bias_final,bias_drop
0,ARIMA,2013,0.024469,0.993606,0.800486,0.193120,-1.0,0.043853,0.990559,0.608826,0.381733,-1.0,-0.095999,-0.000717,0.053995,-0.054712
1,ARIMA,2014,0.001348,0.995475,0.811550,0.183926,-1.0,-0.035996,0.993396,0.665360,0.328036,-1.0,0.164512,0.000910,0.043052,-0.042143
2,ARIMA,2015,0.033451,0.995947,0.745181,0.250766,-1.0,0.042241,0.994314,0.584369,0.409944,-1.0,-0.012728,0.001628,0.104698,-0.103070
3,ARIMA,2016,-0.040246,0.994068,0.815479,0.178589,-1.0,-0.020547,0.988831,0.637108,0.351723,-1.0,0.076062,-0.000369,-0.028030,0.027661
4,ARIMA,2017,0.318239,0.980389,0.517551,0.462838,-1.0,0.768113,0.969957,-0.103737,1.073694,-1.0,0.366589,-0.007793,-0.511570,0.503777
5,HLSTM-PBHM,2013,-0.049092,0.885402,0.867373,0.018029,17.0,-0.094941,0.899400,0.853391,0.046009,5.0,-6.750503,8.995498,9.854489,-0.858991
6,HLSTM-PBHM,2014,0.004484,0.974272,0.897135,0.077137,4.0,-0.028029,0.959397,0.931851,0.027546,4.0,-4.332876,1.554892,3.262140,-1.707248
7,HLSTM-PBHM,2015,0.028699,0.917717,0.928633,-0.010916,9.0,0.041159,0.915641,0.862884,0.052758,6.0,1.187701,-2.299696,-1.014208,-1.285488
8,HLSTM-PBHM,2016,0.109108,0.942600,0.822672,0.119929,35.0,0.071688,0.952767,0.817647,0.135120,3.0,-4.346118,4.432567,5.609102,-1.176534
9,HLSTM-PBHM,2017,0.054209,0.918261,0.835579,0.082683,2.0,0.129879,0.864770,0.635077,0.229694,2.0,8.314560,-2.814179,-11.304312,8.490133


In [9]:
# calculate averaged metrics of hydrologic model
dfh = pd.read_csv(DATA_PATH, parse_dates=["time"])
hyd_metrics = {}
for year in range(2012, 2018):
      dfp = dfh.loc[dfh["time"].dt.year == year]
      hyd_metrics[year] = {
          "kge":  calculate_kge(dfp.qmeasval.values, dfp.qsim.values),
          "nse":  calculate_nse(dfp.qmeasval.values, dfp.qsim.values),
          "bias": calculate_bias(dfp.qmeasval.values, dfp.qsim.values),
      }    
    
# calculate averaged metrics of models
evalu = {}
for n, key in enumerate(models.keys()):
    metrics = load_metrics(os.path.join(models[key].lg_path, "metrics.txt"))
    evalu[key] = {}
    for i in range(0,5):
        year = 2013 + i
        evalu[key][year] = {}
        for j, met in enumerate(["kge", "nse", "bias"]):
            metric = metrics["test"][met][i]
            evalu[key][year][met] = metric
            
# create dataframe to sumup results
years = np.array([x for x in range(2013,2018)])
metric_names = ["kge", "nse", "bias"]

df_avg_model = pd.DataFrame(index=years)

for year in years:
    for met in metric_names:
        df_avg_model.loc[year, f"hyd_{met}"] = hyd_metrics[year][met]

for n,key in enumerate(evalu.keys()):
    for year in years:
        for met in metric_names:
            df_avg_model.loc[year, f"{key}_{met}"] = np.mean(evalu[key][year][met])

df_avg_model   

,hyd_kge,hyd_nse,hyd_bias,arima_kge,arima_nse,arima_bias,lstm_pbhm_kge,lstm_pbhm_nse,lstm_pbhm_bias,lstm_kge,lstm_nse,lstm_bias
2013,0.631525,0.185013,-15.838105,0.841647,0.686484,-0.020626,0.871407,0.881400,9.099347,0.895088,0.868762,8.273931
2014,0.738640,0.494420,-15.556921,0.851671,0.739703,0.031338,0.928595,0.944025,3.282738,0.945149,0.944854,-4.540818
2015,0.505049,0.236257,-6.201059,0.820567,0.701578,0.069297,0.942643,0.907690,-1.794253,0.820505,0.832941,-6.223842
2016,0.738958,0.512468,-10.459236,0.854126,0.714077,-0.017186,0.855134,0.860518,5.310024,0.878713,0.884305,-5.672969
2017,0.191555,-4.243535,-59.985891,0.564865,0.022320,-0.326514,0.866526,0.743704,-8.058480,0.872876,0.804207,2.680256


In [10]:
# write to txt file in latex format
mask = get_bold_mask(df_avg_model.abs(), [np.argmax, np.argmax, np.argmin], 3, 0)
df2latex(df_avg_model[df_avg_model.columns[[0,1,2,3,4,5,6,7,8,9,10,11]]], os.path.join(PLOT_PATH, r"table_2_mean_metrics.txt"), mask, '6.2f')

plots\table_2_mean_metrics.txt


# Table 3: Generalization

In [11]:
# rearange df created for table 2 to show generalization errors
df.groupby("name").mean()[["kge_general", "nse_general", "bias_general"]]
dfgen = df.set_index("name")[["year","kge_general", "nse_general", "bias_general"]]
df1 = dfgen.loc["ARIMA",:].set_index("year")
df1.columns = [x+"_ARIMA" for x in df1.columns]
df2 = dfgen.loc["HLSTM",:].set_index("year")
df2.columns = [x+"_HLSTM" for x in df2.columns]
df3 = dfgen.loc["HLSTM-PBHM",:].set_index("year")
df3.columns = [x+"_HLSTM-PBHM" for x in df3.columns]

df_general = pd.concat([df1, df3, df2], axis=1)
df_general

,kge_general_ARIMA,nse_general_ARIMA,bias_general_ARIMA,kge_general_HLSTM-PBHM,nse_general_HLSTM-PBHM,bias_general_HLSTM-PBHM,kge_general_HLSTM,nse_general_HLSTM,bias_general_HLSTM
year,,,,,,,,,
2013,0.024469,0.043853,-0.095999,-0.049092,-0.094941,-6.750503,-0.294380,-0.205412,4.751613
2014,0.001348,-0.035996,0.164512,0.004484,-0.028029,-4.332876,0.015144,-0.022472,4.805260
2015,0.033451,0.042241,-0.012728,0.028699,0.041159,1.187701,0.155068,0.118759,6.900319
2016,-0.040246,-0.020547,0.076062,0.109108,0.071688,-4.346118,0.085560,0.045406,5.044027
2017,0.318239,0.768113,0.366589,0.054209,0.129879,8.314560,0.048712,0.088745,-2.436095


In [12]:
# write to txt file in latex format
mask = get_bold_mask(df_general.abs(), np.argmin, 3, 0)
df2latex(df_general.abs(), os.path.join(PLOT_PATH, r"table_3_gernalization.txt"), mask, '6.3f')

plots\table_3_gernalization.txt


# Table 4: Peak Performance

In [13]:
# number of peaks which are plotted per fold
num_peaks_per_fold = 2

# reload model forecasts
reload = False

In [14]:
idx = -10
dfp = pd.DataFrame(columns= ["name", "year", "peak", "peak_flow","hyd_perr", "hyd_poff"], dtype=float)
dfp = dfp.astype({"name":str, "year":int, "peak":int,"hyd_poff":int})     

for n, key in enumerate(models.keys()):
    idx += 10
    print(key)
    eval_path = os.path.join(models[key].hp_path, "eval_peaks.pckl")
    
    if not os.path.exists(eval_path) or reload == True:
        eval_peaks = []

        # load datamodel
        dm = DataModelCV(DATA_PATH,
               target_name       = models[key].target_name,
               hincast_features  = models[key].feat_hindcast,
               forecast_features = models[key].feat_forecast,
             )

        if models[key].is_external_model:
            overlap_length = 0
            hindcast_length = 96
        else:
            # load trial data
            with open(os.path.join(models[key].hp_path, "trial.json")) as f:
                trial = json.load(f)

            hindcast_length = trial['hyperparameters']['values']['hindcast_length']
            try:
                overlap_length = trial['hyperparameters']['values']['osc_length']
            except:
                overlap_length = 0 

        dm.main(os.path.join(CROSS_INDICES_PATH+f"_{overlap_length}", f"cross_indices_{hindcast_length}.pckl"))

        for n_fold in dm.cross_sets.keys():
            year = 2013 + n_fold

            # load dataset
            X, y  = dm.getDataSet(dm.cross_sets[n_fold]["test"], scale=True) 

            # get hydrologial model 
            s = dm.getFeatureSet(n_fold+2, "qsim")[2]
            df = pd.DataFrame({'index':dt(s.index), 'qhyd':s.values}).set_index("index")

            # add ground truth
            s = dm.getFeatureSet(n_fold+2, "qmeasval")[2]
            s.index = dt(s.index)
            df = df.merge(s.rename("qmeas").to_frame(), left_index=True, right_index=True)

            if models[key].is_external_model:
                
                ext_df = pd.read_pickle(os.path.join(models[key].hp_path, f"forecast_{year}.pckl"))
                
                forecasts_df = ext_df[[f"fc{x:d}" for x in range(96)]].copy()
                forecasts_df.columns = [f"q{x:d}" for x in range(96)]

                del ext_df
                
            else:
                # load model
                tf.keras.backend.clear_session()
                model  = tf.keras.models.load_model(os.path.join(models[key].hp_path, f"model_fold_{n_fold:d}.keras"),
                                               custom_objects={'peak_loss': peak_loss,
                                                              'kge_nse_loss':peak_loss})

                yp = model.predict(X, batch_size=1000)
                
                if key == "lstm_residual":
                    _, _, yidx = dm.sets[dm.cross_sets[n_fold]["test"]]
                    simu = dm.getWithIndexArray(["qsim"], yidx)
            
                    # get real values from residuals
                    yp += simu[:,:,0]
            
                forecasts_df = pd.DataFrame(data    = yp, 
                                        columns = [f"q{x:d}" for x in range(yp.shape[1])],
                                        index   = dt(dm.getTimeSet(n_fold+2, 0)[2]))

                # save dataframe 
                df_out = forecasts_df.copy()
                df_out.index = pd.to_datetime(df_out.index, format="%d/%m/%Y %H:%M")
                df_out.to_pickle(os.path.join(models[key].hp_path, f"forecast_{year}.pckl"))
            
            
            # get forcasting stats                          
            for forecast_step in range(1, forecasts_df.shape[1]):
                forecasts_df[f"q{forecast_step:d}"] = forecasts_df[f"q{forecast_step:d}"].shift(forecast_step)
            
            # merge model predctions
            df = df.merge(forecasts_df, left_index=True, right_index=True) 

            # merge prcipitation
            s = pd.Series(dm.getFeatureSet(n_fold+2, "pmean", 0)[2].values, dt(dm.getTimeSet(n_fold+2, 0)[2]))
            df = df.merge(s.rename("pmean").to_frame(), left_index=True, right_index=True)
            
            forecasts_df.dropna(inplace=True)
            stats_df = pd.DataFrame(columns = ["fmin", "fmax", "fmean", 
                                               "fq95", "fq90", "fq75",
                                               "fq50",
                                               "fq25", "fq10", "fq5"],
                                   index = forecasts_df.index)
            
            for i, row in forecasts_df.iterrows():
                stats_df.loc[i] = [row.values.min(), row.values.max(), row.values.mean()] + \
                                        [np.quantile(row.values, float(x[2:])/100) for x in stats_df.columns[3:]]
                  
            # merge stats
            df = df.merge(stats_df, left_index=True, right_index=True)        
            df.dropna(inplace=True)
                                           
            peaks = get_n_peaks(df, "qmeas", num_peaks_per_fold, 24*4)
            peaks["n_fold"] = n_fold + 2

            # add to summary
            eval_peaks.append(peaks)

        # save data to pickle
        df = pd.concat(eval_peaks, axis=0)
        df.to_pickle(eval_path)

    df = pd.read_pickle(eval_path)
    for n_fold in df.n_fold.unique().tolist():
        #print(f"processing fold {n_fold}")
        peaks = df[df.n_fold == n_fold]
        for p in range(num_peaks_per_fold):
            
            dt_index = pd.to_datetime(peaks[peaks.n_peak == p]["qmeas"].index, format="%d/%m/%Y %H:%M")
            timestep = dt_index[1] - dt_index[0]

            
            idx_peak  = peaks[peaks.n_peak == p]["qmeas"].argmax()
            peak_flow = peaks[peaks.n_peak == p]["qmeas"].max()
            
            dfp.loc[idx+n_fold+5*p, ["name", "year", "peak", "peak_flow"]] = [models[key].name, int(2011+n_fold), p, peak_flow]
            
            
            dfp.loc[idx+n_fold+5*p, ["hyd_perr", "hyd_poff"]] = [peaks[peaks.n_peak == p]["qhyd"].max() - peak_flow,
                                                                 peaks[peaks.n_peak == p]["qhyd"].argmax() - idx_peak,
                                                                ]
            dfp.loc[idx+n_fold+5*p, [f"perr_{x}" for x in range(96)]] = [peaks[peaks.n_peak == p][f"q{x}"].max() - peak_flow for x in range(96)]
            dfp.loc[idx+n_fold+5*p, [f"poff_{x}" for x in range(96)]] = [peaks[peaks.n_peak == p][f"q{x}"].argmax() - idx_peak for x in range(96)]
       
dfp = dfp.astype({"year": int, "peak":int,"hyd_poff":int})     
dfp.reset_index(inplace=True, drop=True)

arima
lstm_pbhm
lstm


In [15]:
df_peaks = dfp[["name", "year", "peak", "peak_flow", "hyd_perr", "hyd_poff"]].copy()

df_peaks["peak_median"] = dfp.filter(regex='^perr').median(axis=1)
df_peaks["off_median"]  = dfp.filter(regex='^poff').median(axis=1).astype(int)

df_peaks["hyd_perr"]    = 100 * df_peaks["hyd_perr"]    /  df_peaks["peak_flow"]
df_peaks["peak_median"] = 100 * df_peaks["peak_median"] /  df_peaks["peak_flow"]

df = df_peaks.loc[0:9, df_peaks.columns[1:6]]

df = df.join(df_peaks.loc[0:9,   df_peaks.columns[6:8]].reset_index(drop=True), how='left', rsuffix="arima")
df = df.join(df_peaks.loc[9:18,  df_peaks.columns[6:8]].reset_index(drop=True), how='left', rsuffix="_hlstm-pbhm")
df = df.join(df_peaks.loc[18:27, df_peaks.columns[6:8]].reset_index(drop=True), how='left', rsuffix="_hlstm")

df = df.set_index("year", drop=True)
df

,peak,peak_flow,hyd_perr,hyd_poff,peak_median,off_median,peak_median_hlstm-pbhm,off_median_hlstm-pbhm,peak_median_hlstm,off_median_hlstm
year,,,,,,,,,,
2013,0,15.00,-90.340227,31,-75.335372,47,-44.686879,29,-16.205046,0
2013,1,10.02,13.313555,20,-3.517414,19,-49.827193,8,17.440398,3
2014,0,7.27,3.475723,18,2.120575,18,-33.452904,-1,-46.141109,7
2014,1,6.23,23.296167,16,21.279349,16,-20.004277,-2,-37.594951,2
2015,0,5.85,-62.460701,36,-46.905234,36,-22.537425,0,-15.893603,-2
2015,1,3.33,4.708865,49,6.196197,48,-9.788796,2,-18.890903,2
2016,0,17.94,-73.571109,26,-47.397975,47,-23.837149,2,23.394184,1
2016,1,9.99,-39.413896,-96,-33.925033,-89,-67.545650,4,-25.434310,2
2017,0,9.21,-49.932621,25,-43.083815,37,-56.386922,63,-62.017450,3


In [16]:
# write to txt file in latex format
mask = get_bold_mask(df.abs(), np.argmin, 2, 2)
df2latex(df, os.path.join(PLOT_PATH, r"table_4_peak_compare_percent.txt"), mask, ['d', '5.2f', '+5.1f', 'd', '+5.1f', 'd', '+5.1f', 'd', '+5.1f', 'd'])

plots\table_4_peak_compare_percent.txt


### Dataframe results
Used in cp 4.2.1 in line 304-309

In [17]:
df.abs().median()

peak                       0.500000
peak_flow                  8.290000
hyd_perr                  44.673259
hyd_poff                  27.000000
peak_median               38.504424
off_median                36.500000
peak_median_hlstm-pbhm    28.645026
off_median_hlstm-pbhm      2.000000
peak_median_hlstm         24.414247
off_median_hlstm           2.000000
dtype: float64